# IMDB Scraper
this notebook is purely for scraping the movie info off of imdb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import string
from concurrent.futures import ThreadPoolExecutor, wait
from random import uniform
import random
import pickle
from sklearn.preprocessing import OneHotEncoder
import os
from sklearn.preprocessing import MultiLabelBinarizer
from copy import deepcopy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
# this will contain the urls for all of the movies in the dataset
all_u = []

# this keeps track of movies already scraped so that ther are no duplicates
completed = []
completed_keys = []

# this is where the info for each movie will be stored as a list of dicts until it can be transformed into a dataframe
basic_rows = []
key_word_rows = []

loc_rows = []

next_page = ''
user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Mozilla/5.0 (Windows NT 10.0; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Mozilla/5.0 (X11; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:107.0) Gecko/20100101 Firefox/107.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0',
    'Mozilla/5.0 (Windows NT 10.0; rv:107.0) Gecko/20100101 Firefox/107.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42',
    'Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'
] 

In [15]:
# this gets all of the urls for all the moves of a specified genre
def get_all_genre_urls(genre, max_movies, dataset):
    genre_url = 'https://www.imdb.com/search/title/?title_type=movie&genres=' + genre + '&explore=title_type,genres&ref_=adv_prv'
    req_cat = requests.get(genre_url)
    soup_cat = BeautifulSoup(req_cat.content, 'html.parser')
    movies = soup_cat.find_all('div', class_='lister-item-image float-left')
    for movie in movies:
        if movie.a['href'] not in dataset:
            dataset.append(movie.a['href'])

    next_page = soup_cat.find('a', class_ = 'lister-page-next next-page')['href']
    
    for num in np.arange(max_movies):
        print(num)
        next_page = get_urls(next_page, dataset)

In [16]:
# gets all the urls from one page of movies
def get_urls(page_url, data):
    req = requests.get('https://www.imdb.com' + page_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    movies = soup.find_all('div', class_='lister-item-image float-left')
    for movie in movies:
        if movie.a['href'] not in data:
            data.append(movie.a['href'])
    return soup.find('a', class_ = 'lister-page-next next-page')['href']

In [6]:
# gets all of the info for one movie page
def get_movie(movie_url):
    if movie_url in completed:
        return 0

    time.sleep(uniform(0,2))
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent, 'Accept-Language': 'en-US,en;q=0.5'} 
    req = requests.get('https://www.imdb.com/' + movie_url, headers = headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    if str(req) != '<Response [200]>':
        return 0

    try:
        title = soup.find('h1', class_ = 'sc-b73cd867-0 eKrKux').text
    except:
        try: 
            title = soup.find('h1', class_ = 'sc-b73cd867-0 fbOhB').text
        except:
            try:
                title = soup.find('h1', class_ = 'sc-b73cd867-0 cAMrQp').text
            except:
                title = 'none'
    try:
        wr = soup.find_all('ul', role = "presentation", class_ = 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt')
        try:
            writers = [w.text.split('(')[0] + ' ' + w.a['href'].split('nm')[1].split('/?')[0] for w in wr[1].findAll('li')]
        except:
            writers = ''
        try:
            director = [w.text.split('(')[0] + ' ' + w.a['href'].split('nm')[1].split('/?')[0] for w in wr[0].findAll('li')]
        except:
            director = ''
    except:
        writers = ''
        director = ''
    
#     #try:
#       #  description = soup.find('span', class_ = 'sc-16ede01-2 gXUyNh').text
#    # except:
#        # description = 'no description'
    
    try:
        avg_review = soup.find('span', class_="sc-7ab21ed2-1 jGRxWM").text
    except:
        avg_review = 20
    
    try:
        raw = soup.find('div', class_="sc-7ab21ed2-3 dPVcnq").text
        if raw[-1] == 'M':
            num_reviews = float(raw[:-1]) * 1000000
        elif raw[-1] == 'K':
            num_reviews = float(raw[:-1]) * 1000
        else:
            num_reviews = float(raw)
    except:
        num_reviews = 0
#     #try:
#         #poster = soup.find('div', class_ = 'ipc-media ipc-media--poster-27x40 ipc-i
#         #mage-media-ratio--poster-27x40 ipc-media--baseAlt ipc-media--poster-l ipc-p
#         #oster__poster-image ipc-media__img').img['src']
#     #except:
#         #poster = 'none'
    
    try:
        budget = int(soup.find('button', text = 'Budget').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    except:
        try:
            budget = int(soup.find('button', text = 'Budget').findNext('div').text[4:].split('(')[0].split()[0].replace(',', ''))
        except:
            budget = 0
        
#    # try:
#       #  dom_gross = int(soup.find('button', text = 'Gross US & Canada').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
#    # except:
#     #    dom_gross = 0
        
#     #try:
#         #world_gross = int(soup.find('button', text = 'Gross worldwide').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
#     #except:
#         #world_gross = 0
        
#     #try:
#         #meta_score = int(soup.find('span', class_ = 'score-meta').text)
#     #except:
#         #meta_score = 0
    
    try:
        rating = soup.find_all('a', class_ = 'ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh')[1].text
    except:
        rating = 'UNK'
        
    try:
        date = soup.find('a', text = 'Release date').findNext('div').text.partition('(')[0].strip()
    except:
        date = '?'
        
    try:
        country = [soup.find('button', text = 'Country of origin').findNext('div').text]
    except:
        try:
            countries_raw = soup.find('button', text = 'Countries of origin').next_sibling()[0].findAll('li')
            country = [c.text for c in countries_raw]
        except:
            country = ['unknown']
    
    genres = []
    try:
        gs = soup.find_all('a', class_ ='sc-16ede01-3 bYNgQ ipc-chip ipc-chip--on-baseAlt')
        for g in gs:
            genres.append(g.text)
    except:
        genres = ['none']
       
    all_comps = []
    try:
        comps = soup.find('a', text = 'Production companies').findNext('div').find_all('li')
        for comp in comps:
            all_comps.append(comp.text)
    except:
        all_comps = ['none']
    
    all_cast = []
    try:
        try:
            cast = soup.findAll('a', class_ = 'sc-bfec09a1-1 gfeYgX')[:10]
        except:
            cast = soup.findAll('a', class_ = 'sc-bfec09a1-1 gfeYgX')
            
        for act in cast:
            try:
                all_cast.append(act.text + ' ' + act['href'].split('nm')[1].split('/?')[0])
            except:
                all_cast.append('no info')
    except:
        all_cast = ['none']
    
    try:
        runtime = soup.find('button', text = 'Runtime').findNext('div').text.split()
        run = int(runtime[0]) * 60
        if len(runtime) > 2:
            run = run + int(runtime[2])
    except:
        run = 0
    
    all_langs = []
    try:
        langs = soup.find('button', text = 'Languages').findNext('div').ul.find_all('li')
        for lang in langs:
            all_langs.append(lang.text)
    except:
        try:
            all_langs.append(soup.find('button', text = 'Language').findNext('div').ul.li.text)
        except:
            all_langs = ['none']
    
    # this will be put into a dataframe later
    basic_rows.append({'id' : movie_url[9:-1], 'title': title, 'writer': writers, 'director': director, 'actors': all_cast,
                       'num_review': num_reviews,'release_date': date, 'country': country,# 'description': description, 'm_rating': rating,
                          'imdb_rating': avg_review, #'meta_score': meta_score, #'poster': poster, 
                          'languages': all_langs, 'runtime': run, 'genres': genres, 'budget': budget,
                          # 'dom_gross': dom_gross, 'world_gross': world_gross,
                          'prod_comp': all_comps})
    print(len(basic_rows), title, end = '\r')
    completed.append(movie_url)

In [7]:
# gets the keywords for an individual movie
def get_keys(movie_url):
    if movie_url in completed_keys:
        return 0

    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent, 'Accept-Language': 'en-US,en;q=0.5'} 
  
    req = requests.get('https://www.imdb.com/' + movie_url + 'keywords', headers = headers)
    soup = BeautifulSoup(req.content, 'html.parser')

    if str(req) != '<Response [200]>':
        return 0

    all_words = []
    try:
        words = soup.find_all('div', class_ = 'sodatext')
        
        for word in words:
            all_words.append(word.text.strip())
    except:
           pass
    
    key_word_rows.append({'id': movie_url[9:-1], 'key_words': all_words})
    print(len(key_word_rows))
    completed_keys.append(movie_url)

In [ ]:
# this cell gets all of the movies from every genre
get_all_genre_urls('comedy', 2080, all_u)
with open('comedy.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('sci-fi', 325, all_u)
with open('sci-fi.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('horror', 720, all_u)
with open('horror.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('romance', 1030, all_u)
with open('romance.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('action', 1035, all_u)
with open('action.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('thriller', 1050, all_u)
with open('thriller.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('drama', 4440, all_u)
with open('drama.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('mystery', 366, all_u)
with open('mystery.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('crime', 700, all_u)
with open('crime.pkl', 'wb') as f:
    pickle.dump(all_u, f)

get_all_genre_urls('animation', 160, all_u)
with open('animation.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('adventure', 500, all_u)
with open('adventure.pkl', 'wb') as f:
    pickle.dump(all_u, f)

get_all_genre_urls('fantasy', 335, all_u)
with open('fantasy.pkl', 'wb') as f:
    pickle.dump(all_u, f)

#get_all_genre_urls('biography', 160, all_u)
get_all_genre_urls('family', 335, all_u)
with open('family.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('history', 174, all_u)
with open('history.pkl', 'wb') as f:
    pickle.dump(all_u, f)

get_all_genre_urls('war', 193, all_u)
with open('war.pkl', 'wb') as f:
    pickle.dump(all_u, f)
    
get_all_genre_urls('western', 179, all_u)
with open('western.pkl', 'wb') as f:
    pickle.dump(all_u, f)

In [12]:
# save the master list of movie urls
with open('master_url_list.pkl', 'wb') as f:
    pickle.dump(all_u, f)

In [13]:
len(all_u)

421639

In [28]:
# gets the data from all of the movie urls in all_u
def get_movie_data(urls, start, finish):
#     #key_word_rows.clear()
    basic_rows.clear()

    with ThreadPoolExecutor(max_workers = 10) as executor:
        futures = [executor.submit(get_movie, u) for u in all_u[start:finish]]
        wait(futures)
        print(len(basic_rows))
        print('thread done')
    
    print(len(basic_rows))

    print('done2')

In [ ]:
def get_key_data(urls, start, finish):
    key_word_rows.clear()

    with ThreadPoolExecutor(max_workers = 10) as executor:
        futures = [executor.submit(get_keys, u) for u in all_u[start:finish]]
        wait(futures)
        print('thread done')

In [29]:
get_movie_data(all_u, 0, 421630)
get_key_data(all_u, 0, 421630)

21624 Apocalypse CowboyGolden Westeersrsrairie (Das Vermachtnis der Prarie)ear Bakersfieldnskikh sovietskikh zemel
thread done
21624
done2


In [17]:
# converts the rows into a df
df = pd.DataFrame(columns = ['id', 'title', 'release_date', 'runtime', 'm_rating', 'imdb_rating', 'num_review', 'director', 
                             'writer', 'actors', 'genres' , 'country', 'languages', 'prod_comp', 'budget'])
for row in master_basic_rows:
    print(len(df), end = '\r')
    df = df.append(row, ignore_index = True)

In [ ]:
df_key = pd.DataFrame(columns = ['id', 'key_words'])
for row in key_word_rows:
    print(len(df_key), end = '\r')
    df_key = df_key.append(row, ignore_index = True)

In [4]:
# this cell merges the keys and basic rows to get the final df
films_df = pd.merge(df, all_keys, how = 'outer', on = 'id')
films_df = films_df.dropna(subset = 'title')

In [5]:
# this cell is all bout getting rid of the movies that don't have enough data for me to work with

films = deepcopy(films_df)
films = films.drop_duplicates(subset= ['id'])
films = films[films.release_date != '?']
films = films[films.runtime != 0]
films.release_date = pd.to_datetime(films.release_date)
films.release_date = films.release_date.map(lambda x: x if x.year < 2023 else 2099)
films = films[films.release_date != 2099]
#films.release_date = films.release_date.apply(lambda x: x.toordinal())
films = films[films.imdb_rating != 20]
#films.locations = films.locations.map(lambda x: [l.split(',')[-1] for l in x])
films = films[films.num_review > 100]
films[['runtime', 'num_review', 'budget']] = films[['runtime', 'num_review', 'budget']].astype(float)
films.imdb_rating = films.imdb_rating.astype(float)
#films = films[films.budget != 0]
films.dropna(inplace = True)
films.key_words = films.key_words.map(lambda x: [w.lower() for w in x])
films = films.reset_index().drop('index', axis =1)

In [13]:
with open('all_genres_df.pkl', 'wb') as f: 
    pickle.dump(films.drop('budget', axis = 1), f)

In [14]:
films

,id,title,release_date,runtime,m_rating,imdb_rating,num_review,director,writer,actors,genres,country,languages,prod_comp,budget,key_words
0,10304142,Amsterdam,2022-10-07 00:00:00,134.0,R,6.1,36000.0,[David O. Russell],[David O. Russell],"[Christian Bale, Margot Robbie, John David Was...","[Comedy, Drama, History]","[United States, Japan]","[English, French]","[20th Century Studios, Canterbury Classic, For...",80000000.0,"[conspiracy, 1930s, fascism, war wound, year 1..."
1,12593682,Bullet Train,2022-08-05 00:00:00,127.0,R,7.3,215000.0,[David Leitch],"[Zak Olkewicz, Kôtarô Isaka]","[Brad Pitt, Joey King, Aaron Taylor-Johnson, B...","[Action, Comedy, Thriller]","[Japan, United States]","[English, Japanese, Spanish, Russian]","[87North, CTB Inc., Hill District Media]",85900000.0,"[train, assassin, russian, briefcase, fight on..."
2,9764362,The Menu,2022-11-18 00:00:00,107.0,R,7.5,40000.0,[Mark Mylod],"[Seth Reiss, Will Tracy]","[Ralph Fiennes, Anya Taylor-Joy, Nicholas Houl...","[Comedy, Horror, Thriller]",[United States],"[Spanish, English]","[Alienworx Productions, Alienworx Productions,...",35000000.0,"[restaurant, psychological horror, cooking, ch..."
3,12003946,Violent Night,2022-12-02 00:00:00,112.0,R,7.1,15000.0,[Tommy Wirkola],"[Pat Casey, Josh Miller]","[David Harbour, John Leguizamo, Alex Hassell, ...","[Action, Comedy, Crime]","[United States, Canada]",[English],"[87North, Québec Production Services Tax Credit]",20000000.0,"[two word title, santa claus, christmas eve, m..."
4,17220704,A Christmas Story Christmas,2022-11-17 00:00:00,98.0,PG,6.8,9800.0,[Clay Kaytis],"[Jean Shepherd, Nick Schenk, Peter Billingsley]","[Peter Billingsley, Erinn Hayes, River Drosche...","[Comedy, Family]","[United States, Mexico, Hungary, Canada]",[English],"[Warner Bros., Legendary Entertainment, Wild W...",0.0,"[christmas, mexico, hungary, sequel, ralphie p..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36518,0024516,Sagebrush Trail,1933-12-15 00:00:00,3240.0,Passed,5.4,1300.0,[Armand Schaefer],"[Lindsley Parsons, Will Beale]","[John Wayne, Nancy Shubert, Lane Chandler, Yak...",[Western],[United States],[English],[none],0.0,"[cowboy, 1880s]"
36519,0051496,Cowboy,1958-03-23 00:00:00,92.0,Approved,6.7,3200.0,[Delmer Daves],"[Frank Harris, Edmund H. North, Dalton Trumbo]","[Glenn Ford, Jack Lemmon, Anna Kashfi, Brian D...",[Western],[United States],[English],"[Columbia Pictures, Phoenix Pictures]",0.0,"[hotel, cowboy, cattle baron, mexico, cattle r..."
36520,0040835,Station West,1948-09-01 00:00:00,97.0,Approved,6.6,1300.0,[Sidney Lanfield],"[Frank Fenton, Winston Miller, Luke Short]","[Dick Powell, Jane Greer, Agnes Moorehead, Tom...",[Western],[United States],[English],[none],0.0,"[arizona, woman with a gun, arizona territory,..."
36521,0042744,Montana,1950-01-28 00:00:00,76.0,Passed,6.1,1200.0,"[Ray Enright, Raoul Walsh]","[James R. Webb, Borden Chase, Charles O'Neal]","[Errol Flynn, Alexis Smith, S.Z. Sakall, Dougl...",[Western],[United States],[English],[none],2000000.0,"[fiance fiancee relationship, 1880s, sheep, ra..."


In [19]:
# final_driver = webdriver.Chrome()
# login(final_driver)
# time.sleep(25)

# all_names = list(actor_urls.items())
#actor_pop_dict = {}

# for act in all_names[:10000]:
#     name = act[0]
#     url = act[1]
#     actor_pop_dict[name] = get_actor(url, final_driver)
    
# with open('actor_pop_1.pkl', 'wb') as f:
#     pickle.dump(actor_pop_dict, f)
    
for act in all_names[3255: 39000]:
    name = act[0]
    url = act[1]
    actor_pop_dict[name] = get_actor(url, final_driver)
    
with open('actor_pop_4.pkl', 'wb') as f:
    pickle.dump(actor_pop_dict, f)
    
# for act in all_names[20000:30000]:
#     name = act[0]
#     url = act[1]
#     actor_pop_dict[name] = get_actor(url, final_driver)
    
# with open('actor_pop_3.pkl', 'wb') as f:
#     pickle.dump(actor_pop_dict, f)

IndexError: list index out of range

In [9]:
len(actor_urls)

48443

In [ ]:

#os.system('pip3 install rotten_tomatoes_scraper')
#from rotten_tomatoes_scraper.rt_scraper import MovieScraper

#os.system('npm install metacritic-scraper')
#os.system('pip install selenium')
#import selenium

In [ ]:
sc-bfec09a1-1 gfeYgX

In [80]:

user_agent = random.choice(user_agents) 
headers = {'User-Agent': user_agent, 'Accept-Language': 'en-US,en;q=0.5'} 
req = requests.get('https://www.imdb.com/' + '/title/tt0073195/', headers = headers)
soup = BeautifulSoup(req.content, 'html.parser')

In [82]:
wr = soup.find_all('ul', role = "presentation", class_ = 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt')
[w.text.split('(')[0] + w['href'] for w in wr[1].findAll('li')]

KeyError: 'href'

In [93]:
wr[0].a['href'][8:-14]

'0000229/'

In [96]:
get_movie('/title/tt0097576/')

3 Indiana Jones and the Last Crusade


In [97]:
basic_rows[-1]

{'id': '0097576',
 'title': 'Indiana Jones and the Last Crusade',
 'writer': ['Jeffrey Boam 0090151/',
  'George Lucas 0000184/',
  'Menno Meyjes 0583675/'],
 'director': ['Steven Spielberg 0000229/'],
 'actors': ['Harrison Ford 0000148',
  'Sean Connery 0000125',
  'Alison Doody 0233145',
  'Denholm Elliott 0001186',
  'John Rhys-Davies 0722636',
  'Julian Glover 0002103',
  'River Phoenix 0000203',
  'Michael Byrne 0126250',
  'Kevork Malikyan 0539616',
  'Robert Eddison 0248848/']}

In [196]:
base_df.columns[1405:]

Index(['Futurikon', 'G-BASE', 'G. Mahesh Babu Entertainment',
       'G.V. Films Ltd.', 'GAME FREAK', 'GFT Entertainment', 'GH Pictures',
       'GK Films', 'GP Museum Soft', 'Gaga',
       ...
       'Ángel Aranda', 'Ángel Fernández Franco', 'Ángel Ortiz',
       'Ángel del Pozo', 'Ángel Álvarez', 'Édouard Mathé', 'Éric Naggar',
       'Ólafur Darri Ólafsson', 'Óscar Jaenada', 'Ørjan Gamst'],
      dtype='object', length=36920)